In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model
from tensorflow.keras.layers import GlobalMaxPooling1D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

df1 = pd.read_csv("pisci.csv")
df1.rename(columns={ '0':'tekst', '1':'pisac'}, 
                 inplace=True)

#funkcija sample sluzi za nasumicno mesanje samplova  
df1 = df1.sample(frac=1)
df1.size

Using TensorFlow backend.


3300

In [2]:
#tf.random.set_seed(1)
df1.head()

,tekst,pisac
953,tumači posvetili potrebnu pažnju sudeći po bib...,andric
567,se na odstojanju i mene držiš na odstojanju do...,selimovic
817,pa dobro da te poslusam a doktore ne zovi pust...,selimovic
140,mostova na levoj obali drine i na desnoj obali...,andric
165,telo zaobilazeći krv kojaje curila sa mesta gd...,andric


In [3]:
# sece na maksimalno 500 karaktera po sample-u
max_len = 500 
# razmatra prvih 10 000 reci
max_words = 10000 

lista = df1['tekst'].to_list()

# import tokenizer, gleda se samo prvih 10 000 reci
tokenizer = Tokenizer(num_words=max_words) 
# fitujemo tokenizer na listu tekstova
tokenizer.fit_on_texts(lista) 
# konvertujemo tekst u sekvence
sekvence = tokenizer.texts_to_sequences(lista) 

word_index = tokenizer.word_index
print('Found %s unique tokens. ' % len(word_index))

 # pravimo array sekvenci sa maksimalnom duzinom 500 
data = pad_sequences(sekvence, maxlen=max_len)
print('Data Shape: {}'.format(data.shape))


Found 63469 unique tokens. 
Data Shape: (1650, 500)


In [4]:
pisac_enkoder = OneHotEncoder()
#ubacujemo data u promenjivu X
X = np.array(data)

#binarizujemo pisce, koji su kategoricki nominalni tip
y = pd.get_dummies(df1['pisac'])
#uzimamo nazive kolona kako bismo posle mogli da vratimo prezimena pisaca
kolone = list(y.columns.values)
#print(kolone)

y = y.to_numpy()

#printamo shape od X i y
print(X.shape, X.dtype)
print(y.shape, y.dtype)

#delimo podatke na trening i test, gde 25% ide na test a ostalo na trening
X_train, X_test, y_train, y_test =\
train_test_split(X, y, test_size=0.25, random_state=42)


(1650, 500) int32
(1650, 2) uint8


In [5]:
text_input_layer = keras.Input(shape=(500,))
embedding_layer = Embedding(max_words, 50)(text_input_layer)
text_layer = Conv1D(256, 3, activation='relu')(embedding_layer)
text_layer = MaxPooling1D(3)(text_layer)
text_layer = Dropout(0.5)(text_layer)

text_layer = Conv1D(256, 3, activation='relu')(text_layer)
text_layer = MaxPooling1D(3)(text_layer)
text_layer = Dropout(0.5)(text_layer)

text_layer = Conv1D(256, 3, activation='relu')(text_layer)
text_layer = MaxPooling1D(3)(text_layer)
text_layer = Dropout(0.5)(text_layer)

text_layer = Conv1D(256, 3, activation='relu')(text_layer)
text_layer = MaxPooling1D(3)(text_layer)
text_layer = Dropout(0.5)(text_layer)

text_layer = Conv1D(256, 3, activation='relu')(text_layer)
text_layer = MaxPooling1D(3)(text_layer)
text_layer = Dropout(0.5)(text_layer)

text_layer = GlobalMaxPooling1D()(text_layer)
text_layer = Dense(256, activation='relu')(text_layer)
output_layer = Dense(2, activation='softmax')(text_layer)
model = Model(text_input_layer, output_layer)
model.summary()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 500)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 500, 50)           500000    
_________________________________________________________________
conv1d (Conv1D)              (None, 498, 256)          38656     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 166, 256)          0         
_________________________________________________________________
dropout (Dropout)            (None, 166, 256)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 164, 256)          196864    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 54, 256)           0     

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='acc', patience=3)
history = model.fit(X_train, y_train, epochs=50, batch_size=128, 
                    callbacks=[early_stop])

Epoch 1/50
10/10 [==============================] - 7s 719ms/step - loss: 0.6885 - acc: 0.5740
Epoch 2/50
10/10 [==============================] - 7s 668ms/step - loss: 0.6786 - acc: 0.5764
Epoch 3/50
10/10 [==============================] - 7s 665ms/step - loss: 0.6560 - acc: 0.5772
Epoch 4/50
10/10 [==============================] - 7s 651ms/step - loss: 0.5366 - acc: 0.6500
Epoch 5/50
10/10 [==============================] - 7s 743ms/step - loss: 0.3620 - acc: 0.8868
Epoch 6/50
10/10 [==============================] - 6s 635ms/step - loss: 0.1879 - acc: 0.9806
Epoch 7/50
10/10 [==============================] - 7s 672ms/step - loss: 0.0190 - acc: 0.9919
Epoch 8/50
10/10 [==============================] - 7s 677ms/step - loss: 3.1603e-04 - acc: 1.0000
Epoch 9/50
 1/10 [==>...........................] - ETA: 0s - loss: 1.8533e-07 - acc: 1.0000

In [ ]:
plt.plot(history.history['acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch

# Uzimamo poslednjih 5 vrsti
hist.tail()

In [ ]:
y_pred = model.predict(X_test)
y_pred1 = np.argmax(y_pred,axis=1)
y_test1 = np.argmax(y_test,axis=1)
print('Preciznost: %.3f' % precision_score(y_true=y_test1, y_pred = y_pred1))
print('Recall skor: %.3f' % recall_score(y_true=y_test1, y_pred = y_pred1))
print('F1: %.3f' % f1_score(y_true=y_test1, y_pred = y_pred1))

In [ ]:

dummies_pred = pd.get_dummies(kolone)

y_pred = pd.DataFrame(data=y_pred, columns=kolone)
y_test = pd.DataFrame(data=y_test, columns=kolone)
y_pred = y_pred.idxmax(axis=1)
y_test = y_test.idxmax(axis=1)

confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print("MATRICA KOFUZIJE")
print(confmat)

In [ ]:
y_pred = np.array(y_pred)
y_test = np.array(y_test)
promasen_tekst = X_test[y_test != y_pred]

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

#promasen_tekst = decoded_review
tacan_pisac = y_test[y_test != y_pred]
promasen_pisac = y_pred[y_test != y_pred]
duzine = [len(promasen_pisac), len(tacan_pisac), len(promasen_tekst), 5]

for i in range(min(duzine)):
    promasen_tekst1 = ' '.join([reverse_word_index.get(kljuc, '') for kljuc in promasen_tekst[i]])
    print(promasen_tekst1)
    print("Promasen pisac: {}".format(promasen_pisac[i]))
    print("Tacan pisac: {}".format(tacan_pisac[i]))
    print("")

if(min(duzine) == 0):
    print("Nema ni jednog promasaja.")

#REFERENCA: https://medium.com/@romannempyre/sentiment-analysis-using-1d-convolutional-neural-networks-part-1-f8b6316489a2

In [ ]:
model.save("model_pisci.h5")